<div class="alert alert-info">
Привет, Алексей! Меня зовут Светлана и я буду проверять твой проект. Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе. Предлагаю общаться на «ты». Но если это не удобно - дай знать, и мы перейдем на «вы».

<div class="alert alert-success">
<b>👍 Успех:</b> Зелёным цветом отмечены удачные и элегантные решения, на которые можно опираться в будущих проектах.
</div>
<div class="alert alert-warning">
<b>🤔 Рекомендация:</b> Жёлтым цветом выделено то, что в следующий раз можно сделать по-другому. Ты можешь учесть эти комментарии при выполнении будущих заданий или доработать проект сейчас (однако это не обязательно).
</div>
<div class="alert alert-danger">
<b>😔 Необходимо исправить:</b> Красным цветом выделены комментарии, без исправления которых, я не смогу принять проект :(
</div>
<div class="alert alert-info">
<b>👂 Совет:</b> Какие-то дополнительные материалы
</div>
Давай работать над проектом в диалоге: если ты что-то меняешь в проекте по моим рекомендациям — пиши об этом.
Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>🎓 Комментарий студента:</b> Например, вот так.</div>
Пожалуйста, не перемещай, не изменяй и не удаляй мои комментарии. Всё это поможет выполнить повторную проверку твоего проекта быстрее.
 </div>


 ### Описание проекта
 
 Интернет-магазин товаров для дома «Пока все ещё тут» в срочном порядке ищет аналитиков. Вы поможете нашему магазину стать лучше, а клиентам — обустроить дом своей мечты. Наши ближайшие задачи — анализ товарного ассортимента и создание гипотез на основе полученных данных.
«Пока все ещё тут» — мы создаём уют!


 ### Основные задачи исследования:
1. Провести исследовательский анализ данных;
2. Проанализировать торговый ассортимент;
3. Выделить основной и дополнительный ассортимент;
4. Сформулировать и проверить статистические гипотезы.

 ### Описание данных
 Датасет описывает транзакции интернет-магазина товаров для дома и быта «Пока все ещё тут».

Колонки в  *ecommerce_dataset.csv* :

- `date` — дата заказа;
- `customer_id` — идентификатор покупателя;
- `order_id` — идентификатор заказа;
- `product` — наименование товара;
- `quantity` — количество товара в заказе;
- `price` — цена товара.

 ### Предобработка данных


- Проверить типы данных в датафрейме и преобразовать там, где это необходимо.
- Изучить пропуски в датафрейме;
- Проверить, есть ли в данных дубликаты (явные и нет). Проанализировать дубликаты без учёта даты и времени;
- Удалить строки(дубликаты) если все данные в строке совпадают кроме даты;
- Проверить, что одному заказу соответствует один покупатель. Если нет, то оставить первый заказ, остальное удалить;
- Изучить данные на предмет выбросов и аномалий. Если данные не влияют на исследование,то удалить их.

In [ ]:
# импорт необходимых библиотек
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats as st
import matplotlib.pyplot as plt
import datetime as dt
import re
from pymystem3 import Mystem
from collections import Counter
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import plot
from plotly.subplots import make_subplots

In [ ]:
google_id = "1DkqEQWZoHh22YrsYcNUxSzlhgFqdl0tf"
orders = pd.read_csv(f"https://drive.google.com/uc?export=download&id={google_id}" )

In [ ]:
# получаю первоначальную информацию о данных
orders.info()
orders.head(3)

<div class="alert alert-info">
     В данных представлена информация о 7474 заказах. Пропусков нет. Тип столбца с датой не соответствует необходимому.

<div class="alert alert-success">
<b>👍 Успех:</b> Есть описание задачи, план реализации проекта, импортированы нужные библиотеки, датасет загружен и просмотрен
</div>

In [ ]:
# Разделил дату и время на два столбца для упрощения анализа далее.
orders['date'] = pd.to_datetime(orders['date'], format='%Y%m%d%H')
orders = orders.assign(Date=orders.date.dt.date, Time=orders.date.dt.time)
orders.drop(['date'], axis= 1 , inplace= True )

In [ ]:
# Привел названия столбцов к нижнему регистру
orders= orders[['Date','Time','customer_id','order_id','product','quantity','price']]
orders.columns = orders.columns.str.lower()

In [ ]:
# Изменил формат даты на datetime. Часы заказов выделил в отельный столбец. 
orders['date'] = pd.to_datetime(orders['date'])
orders['time'] = pd.to_datetime(orders['time'], format='%H:%M:%S').dt.hour
orders.rename(columns={'time':'hour'}, inplace=True)

In [ ]:
print('Дата последнего заказа:',orders['date'].max())
print('Дата первого заказа:',orders['date'].min())
print('Всего заказов за период:',orders['order_id'].nunique())
print('Всего покупателей за период:',orders['customer_id'].nunique())
print('Уникальных продуктов:',orders['product'].nunique())

In [ ]:
orders[orders.duplicated()].count()

<div class="alert alert-info">
    Явных дубликатов не выявлено.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
orders_without_date = orders.drop(['date','hour'], axis=1)
orders_without_date[orders_without_date.duplicated()].count()

In [ ]:
# Убедился, что есть полное совпадение в данных без даты и времени.
orders_without_date[orders_without_date.duplicated(keep=False)].head(30)

<div class="alert alert-info">
Выявлено 1864 строки с полным совпадением кроме даты. 

In [ ]:
# Удалил строки-дубликаты без учёта даты
orders2 = orders.drop_duplicates(subset=['customer_id', 'order_id','product','quantity','price'])
orders2 = orders2.reset_index()
orders2 = orders2.drop(['index'], axis=1)
print(len(orders2.index))


In [ ]:
# проверил, что строки-дубликаты удалены.
orders_without_date2 = orders2.drop(['date','hour'], axis=1)
orders_without_date2[orders_without_date2.duplicated()].count()

<div class="alert alert-info">
    Удалил строки с неявными дубликаты.

In [ ]:
orders2.shape[0]/orders.shape[0]*100

<div class="alert alert-info">
    В результате преобразований было удалено около 25% данных.

<div class="alert alert-danger">
<s><b>😔 Необходимо исправить:</b> Перед удалением нужно посмотреть на данные и убедиться, что это действительно дубли, посчитать процент удаленных данных</s>
</div>

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
# сгруппировал df по номерам заказов,оставил только строки с уникальными номерами заказов. 
sorted = orders2.sort_values(['order_id', 'customer_id'], ascending = [True, False])
unique_orders = sorted.groupby('order_id').first().reset_index()
unique_orders.info()

<div class="alert alert-info">
    Было выявлено, что некоторым заказам соответсвуют несколько покупателей. Удалил строки с дубликатами покупателей. Количество строк в обновленном датафрейме соответствует количеству заказов в исходном. 

In [ ]:
#проверил,что одному заказу соответствует 1 покупатель.
a = unique_orders.groupby('order_id')['customer_id'].nunique()
a = pd.DataFrame({'order_id':a.index, 'customer_id':a.values})
print(a.sort_values(by='customer_id',ascending=False).head(3))
a['customer_id'].unique()

<div class="alert alert-info">
29 заказов имеют более одного покупателя.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
# Добавил в df столбцы с названиями дня недели и месяца.
unique_orders['day_of_week'] = unique_orders['date'].dt.day_name()
unique_orders['name_of_month'] = unique_orders['date'].dt.month_name()

In [ ]:
# Добавил в df столбцы с номерами дня недели и месяца.
unique_orders['num_of_week'] = unique_orders['date'].dt.weekday
unique_orders['num_of_month'] = unique_orders['date'].dt.month

In [ ]:
# убрал цифры,знаки и английские буквы в столбце product для увеличения скорости леммтизации.
unique_orders['product2'] =  unique_orders['product'].apply(lambda x: re.sub(r'[^А-Яа-яd -]', '', str(x)))

In [ ]:
# привел все пробелы к единому стилю.
unique_orders['product2'] = unique_orders['product2'].str.replace('   ', ' ', regex=True)
unique_orders['product2'] = unique_orders['product2'].str.replace('  ', ' ', regex=True)

In [ ]:
# удалил пробелы в конце строк 
unique_orders['product2'] = unique_orders['product2'].str.rstrip()

In [ ]:
# проверил примененные изменения
unique_orders[['product2']].value_counts(normalize= True).head(3)

In [ ]:
# использовал Mystem() для выделения лемм в отдельный столбец
m = Mystem()
unique_orders['lemmas'] = unique_orders['product2'].apply(m.lemmatize)

<div class="alert alert-danger">
<s><b>😔 Необходимо исправить:</b> Почему решил использовать лемматизацию? Если у тебя получилось разобраться с лемматизацией стоит использовать словари а более компактную функцию (без использования такого количетва if). После разбиения на категории нужно проверить сколько товаров в каждой категории</s>
</div>

In [ ]:
# выделил самые часто встречающиеся слова в названиях товаров с помощью Counter()
lemmas = Counter(unique_orders['lemmas'].sum()).most_common()
lemmas

In [ ]:
building = ['пружина','петля','проволочный','инструмент','стремянка','корыто','цинк','строительный','сверло',\
            'сварка','стяжка','набор','измерительный','длина','батарейка','бензин','шило','сметка','уличный','веревка']
kitchen = ['кружка','лоток','сковорода','сотейник','салатник','кастрюля','котел','овощеварка','мантоварка','соковарка','миска',\
           'противень','нож','столовая','столовый','чайный','терка','толкушка','тарелка','термокружок','свч','ложка','банка',\
           'крышка','пищевой','овощ','венчик','хлебница','рыбочистка','бокал','фужер','чайный','форма','картофелемялка',\
           'ножеточка','бутылка','стакан','лезвие','лопатка','ковш','разделочный','термос','сито','плита','пресс','чайник',\
           'эмалированный','заварочный','покрывало','хлопок','салфетка','одеяло','полотенце','подушка','скатерть','плед']
garden = ['рассада','зелень','томат','капуста','перец','огурец','тыква','кабачок','клубника','арбуз','дыня','патиссон',\
          'баклажан','земляника','морковь','петрушка','пряный','лекарственный','цветок','рассада','однолетний','многолетний',\
          'петуния','черенок','крупноцветковый','цвести','садовый','объем','растение','комнатный','домашний' ,'горшок','кашпо',\
          'пеларгония','кассета','цветок','искусственный','композиция','подарочный','открытка','упаковка','новогодний','картина',\
          'фотография','муляж']
home= ['гладильный','сушилка','одежда','вешалка','подкладка','подрукавник','обувь','плечики','одежный','обувной','d',\
       'вантуз','щетка','чехол','коврик','средство','электрический','миксер','фен','паровой','пылесос','весы','светильник']
cleaning = ['ванный','ванна','ванная','туалет','туалетный','унитазный','унитаз','белье','стиральный',\
            'коврик','ковер','вантуз','посуда','мытье','чистка','щетка','мыло','швабра','зубной','кухонный',\
            'круглый','чистить','перчатка','деревянный','тряпка','скребок','ведро','карниз','совок','губка',\
            'антижир','кольцо','мл','чехол','придверный','средство']
keeping = ['сумка','тележка','кофр','хранение','ящик','коробка','корзина','контейнер','таз','полка','комод']

In [ ]:
# распределил слова из названий товаров по категорям с помощью функции.
def lemmas_category(lemmas):
    if any(i in lemmas for i in cleaning):
        return 'Всё для уборки'
    elif any(i in lemmas for i in garden):
        return 'Всё для сада'
    elif any(i in lemmas for i in keeping):
        return 'Всё для хранения'
    elif any(i in lemmas for i in building):
        return 'Всё для стройки'
    elif any(i in lemmas for i in kitchen):
        return 'Всё для кухни'
    else:
        return 'Всё для дома'
    

In [ ]:
unique_orders['product_category'] = unique_orders['lemmas'].apply(lemmas_category)

In [ ]:
unique_orders['product_category'].unique()

In [ ]:
# все товары отнесены к категорям. 
print(unique_orders['product_category'].isna().sum())
# после добавления категорий данные не "потерялись".
print(unique_orders['product_category'].value_counts().sum())

In [ ]:
# удалил ненужные столбцы
unique_orders = unique_orders.drop(['lemmas','product2'], axis=1)

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
unique_orders.groupby('product_category')['order_id'].count().sort_values(ascending=False).reset_index()

<div class="alert alert-info">
    Больше всего товаров в категориях Всё для сада и всё для уборки, меньше всего-в категории всё для стройки.

In [ ]:
# изучил данные после изменений.
unique_orders.info()
print('Дата последнего заказа:',unique_orders['date'].max())
print('Дата первого заказа:',unique_orders['date'].min())
print('Всего заказов за период:',unique_orders['order_id'].nunique())
print('Всего покупателей за период:',unique_orders['customer_id'].nunique())
print('Уникальных продуктов:',unique_orders['product'].nunique())

<div class="alert alert-info">
    Обновленные данные соответсвтуют изначальным параметрам даты,кол-ву заказов. Кол-во покупателей изменилось, так как были выявлены дубликаты. Кол-во уникальных продуктов также изменилось, вероятно, это следствие несоответсвия одному заказу одного покупателя.

In [ ]:
unique_orders.shape[0]/orders.shape[0]*100

<div class="alert alert-info">
    Было удалено 53% данных от изначальных.

<div class="alert alert-danger">
<s><b>😔 Необходимо исправить:</b> После всех преобразовний нужно сразу отслеживать как данные изменились, на сколько процентов уменьшились, итоговое значение строк</s>
</div>

In [ ]:
unique_orders.sort_values(by='price',ascending=False).head(1)

unique_orders.sort_values(by='price').head(1)

<div class="alert alert-info">
Самый дешевый товар: Львиный зев Волшебный ковер, его цена 9 руб.


Самый дорогой: Сушилка уличная Leifheit 85210 LINOMATIC, её цена 14917 руб.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
# Проверил данные через квантили.
unique_orders.quantile ([0.95,0.99])

In [ ]:
# Изучил данные по ценам товаров выше 99 квантиля.
unique_orders.query('price>=6061').sort_values(by='price',ascending=False).head(3)

In [ ]:
# Изучил данные по количествам товаров выше 99 квантиля.
unique_orders.query('quantity>=30').sort_values(by='quantity',ascending=False).head(3)

In [ ]:
# Оставил данные без выброса.
unique_orders = unique_orders.query('quantity<335')

In [ ]:
# Проверил изменения.
unique_orders.quantile ([0.95,0.99])

<div class="alert alert-info">
- В столбцах с количеством и ценой присутсвуют выбросы.
    
- Был найден и удален выброс в количестве товаров: Вантуз с деревянной ручкой, который заказали 1000 штук.  
    
- В столбце с ценой не было найдено выбросов. Самый дорогой товар: Сушилка уличная Leifheit 85210 LINOMATIC, её цена 14917 руб. Это соответсвует реальности.
    

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
# Построил boxplot для цен товаров.
plt.figure(figsize=(15,5))
sns.boxplot(data=unique_orders, x="price", y="product_category")

In [ ]:
# Построил scatterplot количества товаров в заказах.
plt.figure(figsize=(15,5))
sns.scatterplot(x='quantity', y='product_category', data=unique_orders)

In [ ]:
# Построил boxplot для часов дня заказа.
plt.figure(figsize=(15,5))
sns.boxplot(data=unique_orders, x="hour", y="product_category")

<div class="alert alert-info">
1) Стоимость товара в основном не превышает 2000 руб.
    
2) Кол-во заказанных товаров в основном до 50 шт.
    
3) Самое большое количетсво заказов: с 10:00 до 15:00.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

<div class="alert alert-info">
Были добавлены стобцы с названием дня недели(day_of_week), названием времени года(seasons_of_year) и категории продукта (product_category)

In [ ]:
# Проверил изменения.
unique_orders.head(2)

<div class="alert alert-danger">
<s><b>😔 Необходимо исправить:</b> Какие максимальные и минимальные цены? Они соттветствуею реальности? Сколько товаров одного наименования может быть в заказе? Здесь нет странных значений? Для определения аномалий стоит посмотреть 95 и 99 перцентили, построить боксплоты</s>
</div>

 ### Проведите исследовательский анализ данных
- Изучить самые дорогие и дешевые товары;
- Сравнить данные о выручке по категориям и по кол-ву проданных товаров по категориям;
- Провести анализ продаж по дням недели и часам для выявления времени повышенного спроса;
- Изучить сезонность по временам года;
- Провести ABC анализ  по выручке; 
- Выделить основной и дополнительный ассортимент по выручке;
- Провести ABC анализ  для проданных категорий;
- Выделить основной и дополнительный ассортимент по категориям;
- Провести анализ по дням недели и часам исходя из данных ABC тестов.

In [ ]:
# построил таблицу 10 самых дешевые товаров.
sorted_products = unique_orders[['product','price']]
the_cheapest = sorted_products.drop_duplicates()
the_cheapest=the_cheapest.sort_values(by='price',ascending=True).head(10)
the_cheapest=the_cheapest.reset_index(drop= True)
cm = sns.light_palette("green", as_cmap=True)
the_cheapest.style.background_gradient(cmap=cm)


<div class="alert alert-info">
    В топ-3 самых дешевых товаров входят: 'Укроп Обильнолистный 3,0 г', 'Алиссум (лобулярия) Снежные одежды 0,2 г' и 'Львиный зев Волшебный ковер 0,05 г'. 

In [ ]:
# построил таблицу 10 самых дорогих товаров.
sorted_products
most_expensive = sorted_products.drop_duplicates()
most_expensive=most_expensive.sort_values(by='price',ascending=False).head(10)
most_expensive=most_expensive.reset_index(drop= True)
cm = sns.light_palette("green", as_cmap=True)
most_expensive.style.background_gradient(cmap=cm)

<div class="alert alert-info">
    Топ-3 самых дорогих продукта составляют: 'Сушилка уличная Leifheit зеленая	','Сумка-тележка хозяйственная Andersen Royal Shopper, Hera, синяя' и 'Сумка-тележка хозяйственная Andersen Alu Star Shopper, Hava, синяя'

In [ ]:
# Построил визуализации распределения категорий по выручке и кол-ву проданных товаров.
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

sorted_by_revenue = unique_orders.groupby('product_category')['price'].sum().sort_values(ascending=False).reset_index() 
fig = go.Figure(go.Pie(labels=sorted_by_revenue['product_category'], values=sorted_by_revenue['price'], pull=[0.2, 0, 0, 0]))
fig.update_layout(
    title_text="Выручка по категориям")
fig.show()

sorted_by_quantity = unique_orders.groupby('product_category')['product'].count().sort_values(ascending=False).reset_index() 
fig = px.pie(sorted_by_quantity,values='product', names='product_category',color_discrete_sequence=px.colors.sequential.RdBu,hole=0.5)
fig.update_layout(
    title_text="Количество проданных товаров по категориям")
fig.show()


<div class="alert alert-info"> 
-Самой прибыльной является категория Всё для хранения, наименее прибыльная - Всё для сада.

-Больше всего товаров было продано  в категории Всё для сада, меньше всего - в категории Всё для стройки.
    

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
# Создал необходимые для визуализаций df.
unique_orders2 = unique_orders.pivot_table(index='num_of_month',columns='product_category', values='order_id', aggfunc='count').sort_values(by='num_of_month',ascending=False)
unique_orders3 = unique_orders.pivot_table(index='num_of_week',columns='product_category', values='order_id', aggfunc='count').sort_values(by='num_of_week',ascending=False)
unique_orders4 = unique_orders.pivot_table(index='hour',columns='product_category', values='order_id', aggfunc='count')

In [ ]:
# Построил визуализации распределения заказов.
plt.figure(figsize=(15, 5))
fig = sns.lineplot(data=unique_orders2, palette='Paired', dashes=False)
fig.set_title('Распределение заказов по категориям по месяцам', fontsize=10)
fig.set_xlabel('месяц')
fig.set_ylabel('количество заказов')
plt.show()

plt.figure(figsize=(15, 5))
fig = sns.lineplot(data=unique_orders3, palette='Paired', dashes=False)
fig.set_title('Распределение заказов по категориям по дням недели', fontsize=10)
fig.set_xlabel('день недели')
fig.set_ylabel('количество заказов')
plt.show()


plt.figure(figsize=(15, 5))
fig = sns.lineplot(data=unique_orders4, palette='Paired', dashes=False)
fig.set_title('Распределение заказов по категориям по часам', fontsize=10)
fig.set_xlabel('час')
fig.set_ylabel('количество заказов')
plt.show()

<div class="alert alert-info"> 
    
- Наблюдается сезонность в течении года. Самое большое количество заказов зимой,затем кол-во снижается всю весну, летом в июне достигает минимумов и начинает расти к осени, достигая пика в декабре. Данная сезонность соответствует реальности, так как на зиму приходится больше всего выходных и праздников в году.

- Категория 'Всё для уборки' практически во все месяца находится в лидерах по кол-ву заказов.

- Категории 'Всё для стройки' и 'Всё для кухни' на протяжении всего года имеют мЕньшее кол-во заказов.
       
- В течении недели также наблюдается зависимость. На вторник приходится пик числа заказов, затем количество плавно снижается к выходным, достигая дна в субботу.
   
- Наибольшее количество заказов совершается с 10 до 13. Час-пик: с 11 до 12.

<div class="alert alert-danger">
<s><b>😔 Необходимо исправить:</b> Графики стоит построить с разбиенеие на товарные категории, посмотреть на продажи по месяцам, что бы проверить нет ли сезонности</s>
</div>

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
# создал агрегацию для расчета показателей дохода для каждого товара.
abc_revenue = unique_orders.groupby('product').agg(
    unique_product=('product', 'nunique'),
    total_units=('quantity', 'sum'),
    total_revenue=('price', 'sum'),
).sort_values(by='total_revenue', ascending=False).reset_index()
abc_revenue.head(3)

In [ ]:
# добавил столбцы для рассчета ABC классов: кумулятивная выручка,общая выручка и кумулятивная доля в выручке.
# использовал использовать cumsum() для вычисления совокупной суммы выручки, а затем для вычисления текущего процента выручки и сохранения его во фрейме данных
abc_revenue['revenue_cumsum'] = abc_revenue['total_revenue'].cumsum()
abc_revenue['revenue_total'] = abc_revenue['total_revenue'].sum()
abc_revenue['revenue_running_percentage'] = (abc_revenue['revenue_cumsum'] / abc_revenue['revenue_total']) * 100


In [ ]:
# Создал функцию для отнесения каждого товара к соответствующему классу на основе его вклада в процентный доход.
def abc_classify_product(percentage):
       if percentage > 0 and percentage <= 80:
        return 'A'
       elif percentage > 80 and percentage <= 90:
        return 'B'
       else:
        return 'C'

In [ ]:
# Добавил столбцы с категорией ABC и посчитал  рейтинг продаж продукта. 
abc_revenue['abc_class'] = abc_revenue['revenue_running_percentage'].apply(abc_classify_product)

In [ ]:
# Проверил изменения.
abc_revenue.head(2)

In [ ]:
abc_revenue['abc_class'].unique()

<div class="alert alert-danger">
    <s><b>😔 Необходимо исправить:</b> Нужно проверить что получилось</s>
</div>

In [ ]:
# Оставил в abc_revenue для объеденения.
abc_revenue2=abc_revenue[['product','abc_class']]

In [ ]:
# Добавил столбец с abc категорией товара в общий df.
unique_orders_aabbcc = unique_orders.merge(abc_revenue2, how = 'inner', on = 'product')

In [ ]:
# Проверил изменения.
unique_orders_aabbcc.head(2)

In [ ]:
# создал агрегацию для расчета показателей количества для каждого товара.
abc_quantity = unique_orders.groupby('product').agg(
    unique_product=('product', 'nunique'),
    total_units=('quantity', 'count'),
    total_quantity=('quantity', 'sum'),
).sort_values(by='total_quantity', ascending=False).reset_index()
abc_quantity.head(3)

In [ ]:
# добавил столбцы для рассчета ABC классов: кумулятивная выручка,общая выручка и кумулятивная доля в выручке.
# использовал использовать cumsum() для вычисления совокупной суммы выручки, а затем для вычисления текущего процента выручки и сохранения его во фрейме данных
abc_quantity['quantity_cumsum'] = abc_quantity['total_quantity'].cumsum()
abc_quantity['quantity_total'] = abc_quantity['total_quantity'].sum()
abc_quantity['quantity_running_percentage'] = (abc_quantity['quantity_cumsum'] / abc_quantity['quantity_total']) * 100

In [ ]:
# Добавил новый столбец.
abc_quantity['abc_classs'] = abc_quantity['quantity_running_percentage'].apply(abc_classify_product)

In [ ]:
# Проверил изменения.
abc_quantity.head(2)

In [ ]:
# Проверил изменения.
abc_quantity['abc_classs'].unique()

In [ ]:
# Подготовил данные для добавления в основной df.
abc_quantity2=abc_quantity[['product','abc_classs']]

In [ ]:
# Добавил новые данные в основной df
unique_orders_aabbcc = unique_orders_aabbcc.merge(abc_quantity2, how = 'inner', on = 'product')

In [ ]:
# Проверил изменения.
unique_orders_aabbcc.head(2)

In [ ]:
# Объеденил столбцы для получения необходимых abc категорий.
unique_orders_aabbcc['abc_category'] = unique_orders_aabbcc['abc_class']+unique_orders_aabbcc['abc_classs']
unique_orders_aabbcc['abc_category'].unique()

In [ ]:
# Функция для распределения товаров по категориям ассортимента.
def aabbcc_category(unique_orders_aabbcc):
    if unique_orders_aabbcc['abc_category'] =='AA' or\
    unique_orders_aabbcc['abc_category'] =='AB' or\
    unique_orders_aabbcc['abc_category'] =='BA':
        return 'Основной'
    else:
        return 'Дополнительный'

In [ ]:
# Применил функцию к df и добавил новый столбец.
unique_orders_aabbcc['assortment_category']=unique_orders_aabbcc.apply(aabbcc_category, axis=1)

In [ ]:
# Проверил изменения.
unique_orders_aabbcc['assortment_category'].unique()

In [ ]:
# Построил визуализации распределения по категориям ассортимента.
sorted_by_assortment_category = unique_orders_aabbcc.groupby('assortment_category')['product'].count().sort_values(ascending=False).reset_index() 
fig = px.pie(sorted_by_assortment_category,values='product', names='assortment_category',hole=0.5)
fig.update_layout(
    title_text="Распределение количества товаров по категориям ассортимента")
fig.show()
sorted_by_assortment_category2 = unique_orders_aabbcc.groupby('assortment_category')['price'].sum().sort_values(ascending=False).reset_index() 
fig = px.pie(sorted_by_assortment_category2,values='price', names='assortment_category',hole=0.5)
fig.update_layout(
    title_text="Распределение выручки по категориям ассортимента")
fig.show()

<div class="alert alert-info"> 
    Больше всего товаров в дополнительном ассортименте (60%).
 Основной ассортимент приносит 62% выручки компании.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

<div class="alert alert-danger">
<s><b>😔 Необходимо исправить:</b> Разделение нужно делать только по товарам, а не по категориям, в каждой категории могут быть товары и А, и В и С</s>
</div>

In [ ]:
# создал необходимые сводные таблицы для визуализаций ABC теста.
unique_orders4 = unique_orders_aabbcc.pivot_table(index='num_of_month',columns='assortment_category', values='order_id', aggfunc='count').sort_values(by='num_of_month',ascending=False)
unique_orders5 = unique_orders_aabbcc.pivot_table(index='num_of_week',columns='assortment_category', values='order_id', aggfunc='count').sort_values(by='num_of_week',ascending=False)
unique_orders6 = unique_orders_aabbcc.pivot_table(index='hour',columns='assortment_category', values='order_id', aggfunc='count')

In [ ]:
# визуализации ABC теста.

plt.figure(figsize=(15, 5))
fig = sns.lineplot(data=unique_orders4, palette='Paired', dashes=False)
fig.set_title('Продажи по категориям ассортимента по месяцам', fontsize=14)
fig.set_xlabel('месяц')
fig.set_ylabel('количество товаров')
plt.show()
plt.figure(figsize=(15, 5))
fig = sns.lineplot(data=unique_orders5, palette='Paired', dashes=False)
fig.set_title('Продажи по категориям ассортимента по дням недели', fontsize=14)
fig.set_xlabel('день недели')
fig.set_ylabel('количество товаров')
plt.show()
plt.figure(figsize=(15, 5))
fig = sns.lineplot(data=unique_orders6, palette='Paired', dashes=False)
fig.set_title('Продажи по категориям ассортимента по часам', fontsize=14)
fig.set_xlabel('час')
fig.set_ylabel('количество товаров')
plt.show()

<div class="alert alert-info">
- По месяцам наблюдается сезонность-зимой больше всего заказов, летом-меньшее. Почти во все месяцы года количество заказов дополнительного ассортимента превосходит заказы основного.
    
- В декабре и январе самое большое количество заказов во всех категориях ассортимента. В июне наблюдается спад по всем категориям ассортимента. 
    
- Больше всего заказов по понедельникам и вторникам. По субботам спад во всех сегментах.
    
- Часы повышенного спроса: с 10 до 15. Час-пик: с 10 до 12.
   

<div class="alert alert-danger">
<s><b>😔 Необходимо исправить:</b> Такой график стоит смотерть по месяцам, разбивка по временам года не дает нам увидеть сезонность продажи товаров</s>
</div>

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

 ###  Проверьте гипотезы

 ####  Гипотеза 1. 
  ##### нулевая: cредние выручки заказов в будни и выходные равны;
  ##### альтернативная: cредние выручки заказов в будни и выходные различаются. 

Критический уровень статистической значимости равен 5%.

In [ ]:
# Создал столбец с выручкой.
unique_orders['revenue'] = unique_orders['price']*unique_orders['quantity']

In [ ]:
#  Изменил тип столбца с номером дня недели.
unique_orders['num_of_week'] =  unique_orders['num_of_week'].astype(int)

In [ ]:
# Создал функцияю разделения дней недели на части.
def part_of_day (num_of_week):
       if num_of_week >= 6:
        return 'Выходной'
       else:
        return 'Будни'

In [ ]:
# Применил функцию  к df.
unique_orders['part_of_week'] = unique_orders['num_of_week'].apply(part_of_day)

In [ ]:
# Проверил изменения.
unique_orders.head(2)

In [ ]:
# Создал необходимые для t-теста df.
weekday = unique_orders[(unique_orders['revenue'].isna() == False) & (unique_orders['part_of_week'] == "Будни")][['revenue']]
weekends = unique_orders[(unique_orders['revenue'].isna() == False) & (unique_orders['part_of_week'] == "Выходной")][['revenue']]

In [ ]:
# Провелил данные в полученных df.
weekday.head(3)

In [ ]:
# Провелил данные в полученных df.
weekends.head(3)

In [ ]:
#  Для тестирования гипотезы использовал двусторонний t-тест Стьюдента.
alpha = .05
results = st.ttest_ind(weekday, weekends, equal_var=False)
print('p-значение:',results.pvalue)
if results.pvalue < alpha: 
    print("Отвергаем нулевую гипотезу.")
    print("Средние выручки заказов в будни и выходные различаются")
else:
    print("Не получилось отвергнуть нулевую гипотезу.")
    print("Средние выручки заказов в будни и выходные равны")

In [ ]:
# Показал средние выручки заказов в будни и выходные. Это позволяет проверить не ошибся ли я, если они расходятся с тестом Стьюдента.
unique_orders.query('part_of_week == "Выходной"')['revenue'].mean()-unique_orders.query('part_of_week == "Будни"')['revenue'].mean()

<div class="alert alert-info">
    Средние выручки заказов в будни и выходные различаются. Разница составляет 169 руб.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

####   Гипотеза 1.
#####  нулевая: cреднее количество заказанных товаров по выходным и в будни равны;
#####  альтернативная: cреднее количество заказанных товаров по выходным и в будни различаются.

Критический уровень статистической значимости равен 5%.

In [ ]:
# Создал необходимые для t-теста df.
weekday_quantity = unique_orders[(unique_orders['quantity'].isna() == False) & (unique_orders['part_of_week'] == "Будни")][['quantity']]
weekends_quantity = unique_orders[(unique_orders['quantity'].isna() == False) & (unique_orders['part_of_week'] == "Выходной")][['quantity']]

In [ ]:
# Провелил данные в полученных df.
weekday_quantity.head(3)

In [ ]:
# Провелил данные в полученных df.
weekends_quantity.head(3)

In [ ]:
#  Для тестирования гипотезы использовал двусторонний t-тест Стьюдента.
alpha = .05
results = st.ttest_ind(weekday_quantity, weekends_quantity, equal_var=False)
print('p-значение:',results.pvalue)
if results.pvalue < alpha: 
    print("Отвергаем нулевую гипотезу.")
    print("Среднее количество заказанных товаров по выходным и в будни различаются")
else:
    print("Не получилось отвергнуть нулевую гипотезу.")
    print("Среднее количество заказанных товаров по выходным и в будни равны")
   

In [ ]:
# Показал среднее количество заказанных товаров в будни и выходные. Это позволяет проверить не ошибся ли я, если они расходятся с тестом Стьюдента.
unique_orders.query('part_of_week == "Выходной"')['quantity'].mean()-unique_orders.query('part_of_week == "Будни"')['quantity'].mean()

<div class="alert alert-info">
    Среднее количество заказанных товаров по выходным и в будни различаются. Разница составляет примерно 1 заказ.

<div class="alert alert-danger">
<s><b>😔 Необходимо исправить:</b> Нужно сформулировать основную и альтернативную гипотезы и проверить их с помощью статтеста</s>
</div>

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

 ###  Общий вывод
 


Выводы по ассортименту товаров:
- В топ-3 самых дешевых товаров входят: 'Укроп Обильнолистный 3,0 г', 'Алиссум (лобулярия) Снежные одежды 0,2 г' и 'Львиный зев Волшебный ковер 0,05 г';
- Топ-3 самых дорогих продукта составляют: 'Сушилка уличная Leifheit зеленая	','Сумка-тележка хозяйственная Andersen Royal Shopper, Hera, синяя' и 'Сумка-тележка хозяйственная Andersen Alu Star Shopper, Hava, синяя';
- Самой прибыльной является категория Всё для хранения, наименее прибыльная - Всё для стройки;
- Больше всего товаров было продано в категории Всё для сада, меньше всего - в категории Всё для стройки.

Результаты ABC-анализа:
- Все товары были разделены на две категории: основной и дополнительный;
- Больше всего товаров в дополнительном ассортименте (60%);
- Основной ассортимент приносит 62% выручки компании; 
-  По месяцам наблюдается сезонность-зимой больше всего заказов, летом-меньшее. Почти во все месяцы года количество заказов дополнительного ассортимента превосходит заказы основного;
- В декабре и январе самое большое количество заказов во всех категориях ассортимента. В июне наблюдается спад по всем категориям ассортимента;
- Больше всего заказов по понедельникам и вторникам. По субботам спад во всех сегментах;
- Часы повышенного спроса: с 10 до 15. Час-пик: с 10 до 12;
- Средние выручки заказов в будни и выходные различаются;
- Среднее количество заказанных товаров по выходным и в будни различаются.

Основные рекомендации менеджменту:
- увелчить рекламные компании и оповещения пользователей по будням с 10:00 до 15:00. Повышенная активность должна быть в час-пик продаж: с 10:00 до 11:00;
- не предлагать пользователям заказать побольше товаров, а предлагать заказать товар из категории ABC анализа "Основной";
- снизить рекламные компании летом, так как спрос в летние месяца минимален;
- в зимние и осенние месяца увеличить рекламные компании товаров из основного ассортимента.

<div class="alert alert-danger">
    <s><b>😔 Необходимо исправить:</b> Жду презентацию</s>
</div>

[Презентация](https://drive.google.com/file/d/1JeYuCJC6iTXKRjzORAuHKhUCZsmptOX4/view?usp=sharing)


<div class="alert alert-success">
<b>👍 Успех:</b> Хорошая работа, молодец! Все данные исследованы, много визуализации, сделаны выводы по всем разделам, достаточное количество рассуждений и комментариев, в презентации соблюден баланс текстовой и графической информации
</div>

In [ ]:
unique_orders.to_csv('unique_orders.csv', index=False)

In [ ]:
unique_orders.head(2)